In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import sample
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
import random
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import gzip
import json

In [12]:
raw_data = pd.read_csv("Data/Gift_Cards.csv", names=['Item', 'User', 'Rating', 'Timestamp'])
data = raw_data.sample(n=30000)
data.drop(['Timestamp'], axis=1,inplace=True)
data = data[['User', 'Item', 'Rating']]

In [13]:
data

,User,Item,Rating
66700,AR4L02JU87H3O,B00AS527QW,5.0
138630,AGQSBY9YMEW4E,B0091JKJ0M,5.0
116446,A26WX9FT5EF01W,B00PMLD91O,5.0
70362,A3IH95HZIRA03I,B00BWDGY0E,5.0
67085,AUTN6F1IC21UN,B00AS527QW,5.0
...,...,...,...
82692,ARB9WX2DLGLHM,B00CHSWG6O,5.0
62480,ATMY8ZMRNXF12,B00ACA3UB4,5.0
88399,A2IMUJVYCUD8C,B00FTJI60I,4.0
137779,A1DIMQIYIPK49M,B004WKPV3K,5.0


In [14]:
new_df=data.head(10000)
ratings_matrix = new_df.pivot_table(values='Rating', index='User', columns='Item', fill_value=0)
ratings_matrix.head()

Item,B001GXRQW0,B001H53QEO,B001M5GKHE,B002BSHDJK,B002DN7XS4,B002OOBESC,B002UKLD7M,B002VG4AR0,B002XNLC04,B00414Y7Y6,...,B01GF6X842,B01GF6X9UU,B01GF7H0X6,B01GKWEH64,B01GKWEJTO,B01GKWEPBG,B01GKWETLC,B01GKZ3SQG,B01GP1W4LA,B01H5PPJT4
User,,,,,,,,,,,,,,,,,,,,,
A0239841P9KD1PM0XKXT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0435668H73OPGIKAVS2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100297S0KWNIB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100DAK0H0VGXZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A100G8AWDF4572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X = ratings_matrix.T
X1 = X
X.head()

User,A0239841P9KD1PM0XKXT,A0435668H73OPGIKAVS2,A100297S0KWNIB,A100DAK0H0VGXZ,A100G8AWDF4572,A100S8O2ADCUXK,A101QB96637ZIB,A101VAKW3ZAAZR,A10225934GDDWL2ICVX6,A102F85GMNAAH9,...,AZV8WUA9VFDH,AZVDN7JDJWIGA,AZVPQLXP2EGQG,AZW514AE498NS,AZW7TG3CWMRCK,AZWWPKRORAX7G,AZX19SN7VYWE0,AZY609AUO78IN,AZZKM87K3LGIS,AZZZRGMYLGFLM
Item,,,,,,,,,,,,,,,,,,,,,
B001GXRQW0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B001H53QEO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B001M5GKHE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002BSHDJK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B002DN7XS4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(909, 10)

In [17]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(909, 909)

In [18]:
i = X.index[random.randint(0, np.size(correlation_matrix, 0))]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

697

In [19]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(909,)

In [20]:
Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:24]

['B002OOBESC',
 'B002XNLC04',
 'B004KNWWNG',
 'B004KNWWOK',
 'B004KNWWRM',
 'B004KNWWT0',
 'B004KNWWV8',
 'B004KNWWZY',
 'B004KNWX4Y',
 'B004KNWX6M',
 'B004KNWX80',
 'B004LLIL6O',
 'B004LLILAU',
 'B004LLILBY',
 'B004LLILFA',
 'B004LLILG4',
 'B004LLILKK',
 'B004LLILKU',
 'B004LLILM8',
 'B004LLJ5TQ',
 'B004LLJ5UU',
 'B004RD967Y',
 'B004W8D102',
 'B004WKPVOO']

In [21]:
def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_Gift_Cards.json.gz')
data_set = review_data[:30000]
data_set
    

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,"[Gift Cards, Gift Cards]",,[Gift card for the purchase of goods or servic...,,Serendipity 3 $100.00 Gift Card,[],[],,Serendipity 3,[],[],[],{' Product Dimensions: ': '3.4 x 2.1 ...,Grocery,,,,B001BKEWF2
1,"[Gift Cards, Gift Cards]",,[Amazon.com Gift Cards are the perfect way to ...,,Amazon.com Gift Cards,[],[https://images-na.ssl-images-amazon.com/image...,,Amazon,[Amazon.com Gift cards never expire and carry ...,[],"[BT00DC6QU4, B01I4AHZXC, B0719C5P56, B01K8RLHZ...","{'Shipping Weight:': '0.5 ounces', 'Domestic S...",Gift Cards,,,,B001GXRQW0
2,"[Gift Cards, Gift Cards]",,"[<div class=""aplus""> <br>Amazon.com Gift Cards...",,"Amazon.com Gift Cards, Pack of 50 (Old Version...","[B005ESMEBQ, B007EE5WNA, B007EE5OBU, B07HJHK8Y...",[https://images-na.ssl-images-amazon.com/image...,,Amazon,"[Contains 50 Gift Cards, Gift Card has no fees...","1,786 in Gift Cards (","[B005ESMEBQ, B007EE5OBU, B007EE5WNA, B007EE60N...","{'Shipping Weight:': '1.8 pounds (', 'Domestic...",Gift Cards,,,"$1,250.00",B001H53QE4
3,"[Gift Cards, Gift Cards]",,"[<div class=""aplus""> <h4>Amazon.com Gift Cards...",,"Amazon.com $50 Gift Cards, Pack of 50 (Old Ve...",[],[https://images-na.ssl-images-amazon.com/image...,,Amazon,"[Contains 50 Gift Cards, Gift Card has no fees...",[],[],"{'Shipping Weight:': '1.6 pounds', 'Domestic S...",Gift Cards,,,,B001H53QEO
4,"[Gift Cards, Gift Cards]",,[],,Wood Puzzle Magic Box Gift Card,[],[https://images-na.ssl-images-amazon.com/image...,,Creative Crafthouse,[],"[>#2,893,775 in Toys & Games (See Top 100 in T...",[],{},Toys & Games,,,,B001KMWN2K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,"[Gift Cards, Gift Cards]",,"[Hand-Pattied Burgers Chicken, Steaks & Fish ...",,Harvey&#39;s Grill and Bar &amp; Hayden&#39;s...,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"5,808 in Gift Cards (",[],"{'Domestic Shipping: ': 'Currently, item can b...",Gift Cards,,,,B01GMFJ8JU
1543,"[Gift Cards, Gift Cards]",,[<strong>Smoothie King Gift Cards. The Perfect...,,Smoothie King Gift Card,"[B00NVUDIZ0, B00BXLVE6Y, B00O4I5S7O, B00GOLGWV...",[https://images-na.ssl-images-amazon.com/image...,,QuickGifts,"[Over 500 Locations Nationwide!, Smoothies, He...","1,201 in Gift Cards (","[B00NVUDIZ0, B00GOLGWVK, B07JKD8G5X, B00BXLVE6...",{'ASIN:': 'B01GOQIKRI'},Gift Cards,,,$10.00,B01GOQIKRI
1544,"[Gift Cards, Gift Cards]",,"[Cattle Baron Restaurants, Inc.is a privately ...",,Cattle Baron Restaurant Gift Card,"[B00CXZPG0O, B00PG84W32]",[https://images-na.ssl-images-amazon.com/image...,,QuickGifts,"[9 Locations over New Mexico and Texas!, Join ...","3,302 in Gift Cards (",[],{'ASIN:': 'B01GOQHGL4'},Gift Cards,,,,B01GOQHGL4
1545,"[Gift Cards, Gift Cards]",,"[The cards, available in $25 increments up to ...",,Go Play Golf Gift Card,"[B077CZY68Z, B07JJHVWXL, B00NVUDIZ0, B01CZ54L6...",[],,Go Play Golf by Fairway Rewards,[Go Play Golf - the only national gift card th...,"1,015 in Gift Cards (","[B01LYIR4CZ, B077CZY68Z, B07736DNZB, B01N9AE2Z...","{'Shipping Weight:': '0.6 ounces (', 'ASIN:': ...",Gift Cards,,,$50.00,B01GP1W4LA


In [22]:
top25 = pd.DataFrame({'ID': Recommend[0:25]})

In [23]:
top25

,ID
0,B002OOBESC
1,B002XNLC04
2,B004KNWWNG
3,B004KNWWOK
4,B004KNWWRM
5,B004KNWWT0
6,B004KNWWV8
7,B004KNWWZY
8,B004KNWX4Y
9,B004KNWX6M


In [24]:
top25.merge(data_set,left_on = 'ID', right_on = 'asin')

,ID,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
0,B002OOBESC,"[Gift Cards, Gift Cards]",,[Give the furry fun of making a stuffed animal...,,Build-A-Bear Gift Card Collection,[],[],,Build-A-Bear,[Premium greeting cards options are available....,"3,447 in Gift Cards (",[],{'ASIN:': 'B002OOBESC'},Gift Cards,,,,B002OOBESC
1,B002XNLC04,"[Gift Cards, Gift Cards]",,[Outback Steakhouse starts fresh every day to ...,,Outback Steakhouse Gift Card Collection,[],[],,Outback Steakhouse,[Premium greeting cards options are available....,"3,066 in Gift Cards (",[],{'ASIN:': 'B002XNLC04'},Gift Cards,,,,B002XNLC04
2,B004KNWWNG,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Campus Life,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWNG
3,B004KNWWOK,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Queen for a Day,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWOK
4,B004KNWWRM,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - New Baby (Monkey),[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWRM
5,B004KNWWT0,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Happy Birthday (El...,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWT0
6,B004KNWWV8,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Merry Christmas (B...,[],[],,,[],[],"[BT00DC6QU4, BT00DDC7BK, B0719C5P56, B00X4SHPFS]",{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWV8
7,B004KNWWZY,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Happy Father&#39;s...,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWWZY
8,B004KNWX4Y,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - Happy Mother&#39;s...,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWX4Y
9,B004KNWX6M,"[Gift Cards, Gift Cards, No expiration, no fee...",,[],,Amazon Gift Card - Print - New House,[],[],,,[],[],[],{'Domestic Shipping: ': 'Item can be shipped w...,Gift Cards,,,,B004KNWX6M


In [38]:
Top25Recommend = top25.merge(data_set,left_on = 'ID', right_on = 'asin')

In [42]:
Top25Recommend = Top25Recommend[['ID','title']]
Top25Recommend

,ID,title
0,B002OOBESC,Build-A-Bear Gift Card Collection
1,B002XNLC04,Outback Steakhouse Gift Card Collection
2,B004KNWWNG,Amazon Gift Card - Print - Campus Life
3,B004KNWWOK,Amazon Gift Card - Print - Queen for a Day
4,B004KNWWRM,Amazon Gift Card - Print - New Baby (Monkey)
5,B004KNWWT0,Amazon Gift Card - Print - Happy Birthday (El...
6,B004KNWWV8,Amazon Gift Card - Print - Merry Christmas (B...
7,B004KNWWZY,Amazon Gift Card - Print - Happy Father&#39;s...
8,B004KNWX4Y,Amazon Gift Card - Print - Happy Mother&#39;s...
9,B004KNWX6M,Amazon Gift Card - Print - New House
